# Mapping analysis with BART

Analyses:

  - ROI based corr(cosine(AB:CD) for model and CD Match betas)
  - ROI based residual modeling

  - Voxelwise corr(cosine(AB:CD) for model and CD Match betas)

  - Randomise/statistics



In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [12]:
import json

import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

from joblib import Parallel, delayed
import multiprocessing

MAX_CPU = multiprocessing.cpu_count() // 2

In [4]:
from analysis.fmri.analogy_rsa import \
downsample_rdms_df_by_factor, create_models, models_to_df, plotmodels,\
roi_rdm, run_rsa_dfs, subject_rdms, plot_results

from analysis.fmri.analogy_utils import \
    projectSettings, analysisSettings, contrastSettings, order, \
    pu, pa, pv, compile_models, rsa, save_rois, load_rois, load_betas

# load paths and masks
paths = projectSettings["filepaths"]["hoffPaths"]
masks_dict = analysisSettings["roi"]["masks"]


/u/home/n/njchiang/.conda/envs/fmri/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Loading JSON config from config/analyses.json
Loading JSON config from config/contrasts.json
Loading label file from: labels/trialorder_rsa_absorted.csv


## 1. Loading models and data

In [5]:
b = "cope-LSS"
modelnames = ["w2vdiff", "concatword", "rstpostprob9", "rstpostprob79"]
# modelnames = ["mainrel", "subrel", "rel", "humanratings"]

accuracies = pu.load_labels(paths["code"], "labels", "group_accuracy.csv").set_index("Trial")

raw_models_df = pu.load_labels(os.path.join(paths["code"], "labels", "raw_models.csv"))

bart_df = (raw_models_df[::2]
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "rstpostprob79" in c]
            ])  

w2vc_df = (raw_models_df[::2]
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "concatword" in c]
            ])  

w2vd_df = (raw_models_df[::2]
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "w2vdiff" in c]
            ])  



Loading label file from: /u/project/monti/Analysis/Analogy/code/analogy-fmri/labels/group_accuracy.csv
Loading label file from: /u/project/monti/Analysis/Analogy/code/analogy-fmri/labels/raw_models.csv


## Voxelwise correlation


In [13]:
def corrcoef(y, y_pred):
    return np.corrcoef(y, y_pred)[0, 1]

# mask data graymatter
# sub = "sub-01"
maskname = "graymatter-bin_mask"

for sub in projectSettings["subjects"]:
    mask = pu.load_img(paths["root"], "derivatives", sub, "masks", "{}.nii.gz".format(maskname))

    fmri_data, labels, bg_image = load_betas(projectSettings, sub, t="cope-LSS", center=True, scale=False)

    fmri_data = pu.mask_img(fmri_data, mask)
    
    trials = labels[(labels.CD == 1) & (labels.Match == "1")]
    
    fmri_data = fmri_data[trials.index]

    # CD (Match) - AB
    # fmri_data = fmri_data[trials.index] - fmri_data[trials.index - 1]
    # AB and CD
    # fmri_data = fmri_data[labels[labels.Probe == 0].index]
    # CD - AB
    # fmri_data = fmri_data[1::2] - fmri_data[::2]  # double check this

    # Run the accuracy model
    model = "accuracy"
    dists = np.array(
        [accuracies.loc[t]["Group Acc"] for t in trials.TrialTag]
    )
    
    
    results = Parallel(n_jobs=MAX_CPU)(delayed(corrcoef)(fmri_data[:, v], dists) for v in range(fmri_data.shape[1]))
    
#     results = np.array([np.corrcoef(fmri_data[:, v], dists)[0, 1] for v in range(fmri_data.shape[1])])
    results = np.array(results)

    pu.unmask_img(results, mask).to_filename(
        os.path.join(paths["root"], "analysis", sub, 
                     "encoding", "{}-{}-{}_CDMatch.nii.gz".format(sub, model, b)))
    
    # run the rest
    for model in modelnames:
        print("Running {} | {}".format(sub, model))
        # select model this way # keys .isin
        model_df = (raw_models_df[::2]
                    .reset_index(drop=True)
                    .set_index("ABTag")[
                        [c for c in raw_models_df.columns if model in c]
                    ])  
        dists = np.concatenate(
            [rsa.pdist(model_df.loc[
                [t.split("::")[0], t.split("::")[1]]], 
                   metric="cosine") 
             for t in trials.TrialTag])

#         results = np.array([np.corrcoef(fmri_data[:, v], dists)[0, 1] for v in range(fmri_data.shape[1])])
        results = Parallel(n_jobs=MAX_CPU)(delayed(corrcoef)(fmri_data[:, v], dists) for v in range(fmri_data.shape[1]))

    #     results = np.array([np.corrcoef(fmri_data[:, v], dists)[0, 1] for v in range(fmri_data.shape[1])])
        results = np.array(results)
        pu.unmask_img(results, mask).to_filename(
            os.path.join(paths["root"], "analysis", sub, 
                         "encoding", "{}-{}-{}_CDMatch.nii.gz".format(sub, model, b)))
        
    
    

Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/graymatter-bin_mask.nii.gz
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-01_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-02_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-03_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-04_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-05_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-06_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analog

Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/func/sub-06_task-analogy_run-04_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/func/sub-06_task-analogy_run-05_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/func/sub-06_task-analogy_run-06_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/func/sub-06_task-analogy_run-07_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/func/sub-06_task-analogy_run-08_events.csv
Concatenating 8 images
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/reg/BOLD_template.nii.gz
Applying mask
unmasking image...
Running sub-06 | w2vdiff
unmasking image...
Running sub-06 | concatword
unmasking image...
Running sub-06 | rstpostprob9
unmasking image...
Running sub-06 | rstpostprob79
unmaski

Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-11/func/sub-11_task-analogy_run-08_events.csv
Concatenating 8 images
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-11/reg/BOLD_template.nii.gz
Applying mask
unmasking image...
Running sub-11 | w2vdiff
unmasking image...
Running sub-11 | concatword
unmasking image...
Running sub-11 | rstpostprob9
unmasking image...
Running sub-11 | rstpostprob79
unmasking image...
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/masks/graymatter-bin_mask.nii.gz
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/func/sub-12_task-analogy_run-01_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/func/sub-12_task-analogy_run-02_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/func/sub-12_task-analogy_run-03_events.csv
centering image
Load

In [1]:
%%bash
. ~/.bashrc

module load fsl
cd /u/project/monti/Analysis/Analogy

sel=CDMatch
for m in accuracy rstpostprob79 rstpostprob9 w2vdiff concatword
do
  echo ${m}
  for s in 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
  do
    echo sub-${s}
      flirt -in analysis/sub-${s}/encoding/sub-${s}-${m}-cope-LSS_${sel} \
      -out analysis/group/encoding/sub-${s}_${m}_${sel} \
      -ref derivatives/standard/MNI152_T1_2mm_brain.nii.gz \
      -applyxfm -init derivatives/sub-${s}/reg/BOLD_template_to_standard.mat
      fslmaths analysis/group/encoding/sub-${s}_${m}_${sel} -nan analysis/group/encoding/sub-${s}_${m}_${sel}
  done
  fslmerge -t analysis/group/encoding/group-cdmatch_${m}.nii.gz analysis/group/encoding/sub*_${m}_${sel}.nii.gz
  rm analysis/group/encoding/sub*_${m}_${sel}.nii.gz
done


accuracy
sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-10
sub-11
sub-12
sub-13
sub-14
sub-15
sub-16
rstpostprob79
sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-10
sub-11
sub-12
sub-13
sub-14
sub-15
sub-16
rstpostprob9
sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-10
sub-11
sub-12
sub-13
sub-14
sub-15
sub-16
w2vdiff
sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-10
sub-11
sub-12
sub-13
sub-14
sub-15
sub-16
concatword
sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-10
sub-11
sub-12
sub-13
sub-14
sub-15
sub-16


In [2]:
%%bash
. ~/.bashrc

n=1000
module load fsl
cd /u/project/monti/Analysis/Analogy
for m in accuracy rstpostprob79 rstpostprob9 w2vdiff concatword
do
    randomise -i analysis/group/encoding/group-cdmatch_${m} \
      -o analysis/group/encoding/n${n}-CDMatch_${m} \
      -m derivatives/standard/masks/grayMatter \
      -n ${n} -1 -T --uncorrp -v 10
done


randomise options: -i analysis/group/encoding/group-cdmatch_accuracy -o analysis/group/encoding/n1000-CDMatch_accuracy -m derivatives/standard/masks/grayMatter -n 1000 -1 -T --uncorrp -v 10 
Loading Data: 
Data loaded
One-sample design detected; sign-flipping instead of permuting.
65536 sign-flips required for exhaustive test of t-test 1
Doing 1000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting permutation 18
Starting permutation 19
Starting permutation 20
Starting permutation 21
Starting permutation 22
Starting permutation 23
Starting permutation 24
Starting permutation 25
Starting permutation

In [ ]:
# %%bash
# n=1000
# module load fsl
# cd /u/project/monti/Analysis/Analogy
# for m in rstpostprob79 rstpostprob9 w2vdiff concatword
# do
#     randomise -i analysis/group/encoding/group-cdnomatch_${m} \
#       -o analysis/group/encoding/n${n}-CDNoMatch_${m} \
#       -m derivatives/standard/masks/grayMatter \
#       -n ${n} -1 -T --uncorrp -v 10
# done

## 2. ROI analyses

In [6]:
# load ROI data
rois, labels = load_rois(b)

Loading sub-01 betas
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/rois/sub-01_cope-LSS_labels.csv
Loading sub-02 betas
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/rois/sub-02_cope-LSS_labels.csv
Loading sub-03 betas
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-03/rois/sub-03_cope-LSS_labels.csv
Loading sub-04 betas
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-04/rois/sub-04_cope-LSS_labels.csv
Loading sub-05 betas
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-05/rois/sub-05_cope-LSS_labels.csv
Loading sub-06 betas
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/rois/sub-06_cope-LSS_labels.csv
Loading sub-07 betas
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/rois/sub-07_cope-LSS_labels.csv
Loading sub-08 betas
Loading label file from: /u/project/monti/Analysis/Analogy/der

In [7]:
# ROI on CD match

result_dfs = []
subs, masks, models, corrs = [], [], [], []
results = {}

for m in masks_dict:
    results[m] = []
    for sub in projectSettings["subjects"]:
        print("Running {}: {}".format(sub, m))
        label = labels[sub]
        roi = rois[sub][m]
        selector = label[(label.CD == 1) & (label.Match == "1")]
        
        # bart-differences
        dists_bart = np.concatenate(
                        [rsa.pdist(bart_df.loc[
                            [t.split("::")[0], t.split("::")[1]]], 
                               metric="cosine") 
                         for t in selector.TrialTag])

        # w2vC-differences
        dists_w2vc = np.concatenate(
                        [rsa.pdist(w2vc_df.loc[
                            [t.split("::")[0], t.split("::")[1]]], 
                               metric="cosine") 
                         for t in selector.TrialTag])
        # w2vD-differences
        dists_w2vd = np.concatenate(
                [rsa.pdist(w2vd_df.loc[
                    [t.split("::")[0], t.split("::")[1]]], 
                       metric="cosine") 
                 for t in selector.TrialTag])
        
        
        ## Correlations
        subs.append(sub)
        masks.append(m)
        models.append("BART")
        corrs.append(rsa.spearmanr(roi[selector.index].mean(1), dists_bart)[0])
        
        subs.append(sub)
        masks.append(m)
        models.append("W2V-C")
        corrs.append(rsa.spearmanr(roi[selector.index].mean(1), dists_w2vc)[0])
        
        subs.append(sub)
        masks.append(m)
        models.append("W2V-D")
        corrs.append(rsa.spearmanr(roi[selector.index].mean(1), dists_w2vd)[0])
        
        ## Partial correlations
        resids = sm.OLS(roi[selector.index].mean(1), np.vstack([dists_bart]).T).fit().resid
        subs.append(sub)
        masks.append(m)
        models.append("bart-w2v")
        corrs.append(rsa.spearmanr(resids, dists_w2vc)[0])
        
        resids = sm.OLS(roi[selector.index].mean(1), np.vstack([dists_bart]).T).fit().resid
        subs.append(sub)
        masks.append(m)
        models.append("bart-w2vD")
        corrs.append(rsa.spearmanr(resids, dists_w2vd)[0])
        
        resids = sm.OLS(roi[selector.index].mean(1), np.vstack([dists_w2vd]).T).fit().resid
        subs.append(sub)
        masks.append(m)
        models.append("w2vD-bart")
        corrs.append(rsa.spearmanr(resids, dists_bart)[0])
        
        resids = sm.OLS(roi[selector.index].mean(1), np.vstack([dists_w2vc]).T).fit().resid
        subs.append(sub)
        masks.append(m)
        models.append("w2v-bart")
        corrs.append(rsa.spearmanr(resids, dists_bart)[0])
        
        # roi[selector.index].mean(1).shape, dists.shape
result_dfs = pd.DataFrame({"sub": subs, "mask": masks, "model": models, "corr": corrs})
#         print(m, rsa.wilcoxon_onesided(results[m]))
result_dfs.to_csv("/u/home/n/njchiang/transfer/Analogy-CDMatch-regression-replicate.csv")
result_dfs.head()

Running sub-01: BA10m
Running sub-02: BA10m
Running sub-03: BA10m
Running sub-04: BA10m
Running sub-05: BA10m
Running sub-06: BA10m
Running sub-07: BA10m
Running sub-08: BA10m
Running sub-09: BA10m
Running sub-10: BA10m
Running sub-11: BA10m
Running sub-12: BA10m
Running sub-13: BA10m
Running sub-14: BA10m
Running sub-15: BA10m
Running sub-16: BA10m
Running sub-01: BA10l
Running sub-02: BA10l
Running sub-03: BA10l
Running sub-04: BA10l
Running sub-05: BA10l
Running sub-06: BA10l
Running sub-07: BA10l
Running sub-08: BA10l
Running sub-09: BA10l
Running sub-10: BA10l
Running sub-11: BA10l
Running sub-12: BA10l
Running sub-13: BA10l
Running sub-14: BA10l
Running sub-15: BA10l
Running sub-16: BA10l
Running sub-01: BA44
Running sub-02: BA44
Running sub-03: BA44
Running sub-04: BA44
Running sub-05: BA44
Running sub-06: BA44
Running sub-07: BA44
Running sub-08: BA44
Running sub-09: BA44
Running sub-10: BA44
Running sub-11: BA44
Running sub-12: BA44
Running sub-13: BA44
Running sub-14: BA44
Ru

,sub,mask,model,corr
0,sub-01,BA10m,BART,0.041396
1,sub-01,BA10m,W2V-C,-0.075300
2,sub-01,BA10m,W2V-D,0.000535
3,sub-01,BA10m,bart-w2v,-0.090281
4,sub-01,BA10m,bart-w2vD,-0.018252


## 3. Running stats

In [8]:
# ba10l = result_dfs[result_dfs.mask == "BA10l"]
# ba10l.head()
ba10l = result_dfs[result_dfs["mask"] == "BA10l"]
rsa.wilcoxon_onesided(ba10l[ba10l["model"] == "w2vD-bart"]["corr"].values)

0.0006731016748109366

In [9]:
rsa.wilcoxon_onesided(ba10l[ba10l["model"] == "bart-w2v"]["corr"].values)

0.003066684721753846

In [52]:
rsa.wilcoxon_onesided(ba10l[ba10l["model"] == "bart-w2v"]["corr"].values)

0.003066684721753846

accuracy
sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-10
sub-11
sub-12
sub-13
sub-14
sub-15
sub-16


randomise options: -i analysis/group/encoding/group-cdmatch_accuracy -o analysis/group/encoding/n1000-CD_accuracy -m derivatives/standard/masks/grayMatter -n 1000 -1 -T --uncorrp -v 10 
Loading Data: 
Data loaded
One-sample design detected; sign-flipping instead of permuting.
65536 sign-flips required for exhaustive test of t-test 1
Doing 1000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting permutation 18
Starting permutation 19
Starting permutation 20
Starting permutation 21
Starting permutation 22
Starting permutation 23
Starting permutation 24
Starting permutation 25
Starting permutation 26
S

randomise options: -i analysis/group/encoding/group-cdnomatch_rstpostprob79 -o analysis/group/encoding/n1000-CDNoMatch_rstpostprob79 -m derivatives/standard/masks/grayMatter -n 1000 -1 -T --uncorrp -v 10 
Loading Data: 
Data loaded
One-sample design detected; sign-flipping instead of permuting.
65536 sign-flips required for exhaustive test of t-test 1
Doing 1000 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Starting permutation 18
Starting permutation 19
Starting permutation 20
Starting permutation 21
Starting permutation 22
Starting permutation 23
Starting permutation 24
Starting permutation 25
Starti

## Behavioral correlation

In [30]:

# behavioral_averages
# just run spearmanr
from scipy.stats import pearsonr
results = []
trials = order[order.ABMainRel == order.CDMainRel]
# trials = order[order.Probe != 0]

accuracies = pu.load_labels(paths["code"], "labels", "group_accuracy.csv").set_index("Trial")
for model in modelnames:
    print("Running {}".format(model))
    # select model this way # keys .isin
    model_df = (raw_models_df[::2]
                .reset_index(drop=True)
                .set_index("ABTag")[
                    [c for c in raw_models_df.columns if model in c]
                ])  
    dists = np.concatenate(
        [rsa.pdist(model_df.loc[
            [t.split("::")[0], t.split("::")[1]]], 
               metric="cosine") 
         for t in trials.TrialTag])
    
    accs = np.array(
        [accuracies.loc[t]["Group Acc"] for t in trials.TrialTag]
    )
#     results.append(rsa.spearmanr(accs, dists))
    results.append(pearsonr(accs, dists))

results

# behavioral_averages
# just run spearmanr
from scipy.stats import pearsonr
results = []
trials = order[order.ABMainRel != order.CDMainRel]
# trials = order[order.Probe != 0]

accuracies = pu.load_labels(paths["code"], "labels", "group_accuracy.csv").set_index("Trial")
for model in modelnames:
    print("Running {}".format(model))
    # select model this way # keys .isin
    model_df = (raw_models_df[::2]
                .reset_index(drop=True)
                .set_index("ABTag")[
                    [c for c in raw_models_df.columns if model in c]
                ])  
    dists = np.concatenate(
        [rsa.pdist(model_df.loc[
            [t.split("::")[0], t.split("::")[1]]], 
               metric="cosine") 
         for t in trials.TrialTag])
    
    accs = np.array(
        [accuracies.loc[t]["Group Acc"] for t in trials.TrialTag]
    )
#     results.append(rsa.spearmanr(accs, dists))
    results.append(pearsonr(accs, dists))

results

Loading label file from: D:\CloudStation\Grad\Research\montilab-ucla\analogy\labels\group_accuracy.csv
Running w2vdiff
Running concatword
Running rstpostprob9
Running rstpostprob79


[(-0.09629789720831587, 0.2508967260391337),
 (-0.23511954024442838, 0.004557903742077567),
 (-0.24430426881987888, 0.003167977748057288),
 (-0.1521423908369788, 0.06869667669604021)]

## SCRATCH

In [ ]:
pa.rdm(np.array([models["concatword"][v] for v in order["full"].ABTag.values]))

In [ ]:
m = "concatword"
dists = np.array([rsa.pdist(np.vstack([models[m][tt.CDTag], 
                          models[m][tt.ABTag]]), metric="cosine") 
                                     for _, tt in order["full"].iterrows()]).squeeze()
dists

In [ ]:
# fdr output

# ba10
# ba44 + 45
# left aTG mask
# V1
# left IPS


In [ ]:
# run multivariate searchlight regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneGroupOut
modelnames = ["humanratings", "rel","rstpostprob79", "rstpostprob9", "w2vdiff", "concatword"]
analysisSettings["searchlight"]["estimator"] = LinearRegression()
cv = LeaveOneGroupOut()

maskname = "graymatter-bin_mask"
for sub in projectSettings["subjects"].keys():
    imgFile = os.path.join(paths['root'], 'derivatives', sub, 'betas', pu.format_bids_name(sub, 'task-analogy', 'betas-pymvpa.nii.gz'))
    mask = pu.load_img(paths['root'], 
                      'derivatives', sub, 'masks', 
                      maskname + '.nii.gz')
    img = pu.load_img(imgFile)


    labels = pu.load_labels(paths['root'], 
                           'derivatives', sub, 'betas', 
                           pu.format_bids_name(sub, 'task-analogy', 'events-pymvpa.tsv'), 
                           sep='\t', index_col=0)
    conditionSelector = np.where(labels['ab'] == 1)[0]

    img_ab = pu.data_to_img(img.get_data()[:,:,:, conditionSelector], img)
    img_ab.shape
    del img

    these_labels = labels.iloc[conditionSelector]
    sub_results = []
    for m in modelnames:
        this_model = pa.rdm(np.array([models[m][v] for v in these_labels.abtag.values]), metric="cosine")
        if m == "numchar":
            this_model = pa.rdm(np.array([models[m][v] for v in these_labels.abtag.values]))

        result= pa.searchlight_rsa(img_ab, this_model, m=mask)
        sub_results.append(result.scores_)

    pu.data_to_img(np.stack(sub_results, axis=3), img_ab).to_filename(
        os.path.join(paths["root"], "derivatives", sub, "rsa", "{}_{}_searchlight.nii.gz".format(sub, maskname)))
    result = pa.searchlight(img_ab, 
                            these_labels['abmainrel'], m=mask, cv=cv, 
                            groups=these_labels['chunks'],
                            **analysisSettings['searchlight'])
